In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

import matplotlib.pyplot as plt

from misc import HUF_TOKEN
from misc import ALL_ASSIGNMENTS_PATHS, ALL_N_CLUSTERS, FILTER_PATH

from dataset import load_comparison_dataset

In [ ]:
D, _, _, _, _ = load_comparison_dataset(token=HUF_TOKEN)

In [ ]:
all_assignments = [np.load(path) for path in ALL_ASSIGNMENTS_PATHS]

In [ ]:
class ClusteredBradleyTerryModel(nn.Module):
    def __init__(
            self,
            n_items: int,
            n_clusters: int,
            advantage: bool = False,
            reg: float = 1e-2,
    ):
        super(ClusteredBradleyTerryModel, self).__init__()
        self.n_items = n_items
        self.n_clusters = n_clusters
        self.reg = reg
        self.advantage = advantage

        # Bradley-Terry for item-item preferences
        self.bt_item_popularity = nn.Parameter(torch.zeros(n_clusters, n_items))

        # Per-cluster advantage terms
        if advantage:
            self.bt_adv = nn.Parameter(torch.zeros(n_clusters,))

    def predict_proba(
            self,
            cluster_indices,
            item_i_indices,
            item_j_indices,
    ):
        H = self.forward(
            cluster_indices,
            item_i_indices,
            item_j_indices,
        )
        pj = torch.sigmoid(H)
        pi = 1 - pj
        return torch.stack([pi, pj], dim=1)
    
    def forward(
            self,
            cluster_indices,
            item_i_indices,
            item_j_indices,
    ):
        Z_i = self.bt_item_popularity[cluster_indices, item_i_indices]
        Z_j = self.bt_item_popularity[cluster_indices, item_j_indices]  # (n_samples,)
        H = Z_j - Z_i

        if self.advantage:
            H += self.bt_adv[cluster_indices]

        return H
    
    def loss(
            self,
            cluster_indices,
            item_i_indices,
            item_j_indices,
            y,
    ):
        # Log likelihoods
        H = self.forward(
            cluster_indices,
            item_i_indices,
            item_j_indices,
        )
        bt_loss = F.binary_cross_entropy_with_logits(H, y)

        # Regularization
        reg_loss = self.reg * (
            self.bt_item_popularity.norm(p=2).abs()
        ) / self.n_clusters

        return bt_loss + reg_loss

In [ ]:
def fit_cluster_bt(
        cluster_indices: torch.Tensor,
        item_i_indices: torch.Tensor,
        item_j_indices: torch.Tensor,
        y: torch.Tensor,
        bt_max_steps: int = 1000,
        progress_bar: bool = True,
        advantage: bool = False,
):
    n_items = max(item_i_indices.max(), item_j_indices.max()).item() + 1
    assignments = cluster_indices
    n_clusters = cluster_indices.max() + 1

    # Initialize Bradley-Terry model
    model = ClusteredBradleyTerryModel(
        n_items=n_items,
        n_clusters=n_clusters,
        advantage=advantage,
    ).to(item_i_indices.device)

    # Fit Bradley-Terry parameters
    optimizer = torch.optim.LBFGS(model.parameters(), max_iter=bt_max_steps, line_search_fn="strong_wolfe", tolerance_grad=1e-09)

    if progress_bar:
        prog = tqdm(range(bt_max_steps), desc="Fitting BT parameters")

    def closure():
        optimizer.zero_grad()
        loss = model.loss(
            assignments[assignments >= 0],
            item_i_indices[assignments >= 0],
            item_j_indices[assignments >= 0],
            y[assignments >= 0],
        )
        loss.backward()
        if progress_bar:
            prog.update(1)
            prog.set_postfix(loss=loss.item())
        return loss
    
    optimizer.step(closure)

    if progress_bar:
        prog.close()

    return model

In [ ]:
m = max(D["item_i_indices"].max(), D["item_j_indices"].max()).item() + 1
item_i_indices = D["item_i_indices"]
item_j_indices = D["item_j_indices"]
y = D["y"]

def get_pref_matrix(model):
    P = torch.zeros((n_clusters, m, m))
    for c in range(n_clusters):
        item_indices = torch.arange(m)
        cluster_indices = torch.full((m * m,), c, dtype=torch.long)
        item_j_expanded = item_indices.repeat_interleave(m)
        item_i_expanded = item_indices.repeat(m)
        P_c = model.predict_proba(
            cluster_indices,
            item_i_expanded,
            item_j_expanded,
        )[:, 1].view(m, m)
        P[c] = P_c
    return P.detach().numpy()

P0s = []
P1s = []
for n_clusters, assignments in zip(ALL_N_CLUSTERS, all_assignments):
    print(f"Processing n_clusters={n_clusters}...")
    model_0 = fit_cluster_bt(
        cluster_indices=torch.tensor(assignments, dtype=torch.long),
        item_i_indices=D["item_i_indices"],
        item_j_indices=D["item_j_indices"],
        y=D["y"],
        advantage=False,
    )
    model_1 = fit_cluster_bt(
        cluster_indices=torch.tensor(assignments, dtype=torch.long),
        item_i_indices=D["item_i_indices"],
        item_j_indices=D["item_j_indices"],
        y=D["y"],
        advantage=True,
    )
    P0s.append(get_pref_matrix(model_0))
    P1s.append(get_pref_matrix(model_1))

In [ ]:
# get MLEs of preference matrices
PSs = []
for n_clusters, assignments in zip(ALL_N_CLUSTERS, all_assignments):
    assignments = torch.tensor(assignments, dtype=torch.long)
    W = torch.zeros((n_clusters, m, m))
    L = torch.zeros((n_clusters, m, m))
    W.index_put_((assignments, item_i_indices, item_j_indices), (y == 0.0).float(), accumulate=True)
    L.index_put_((assignments, item_i_indices, item_j_indices), (y == 1.0).float(), accumulate=True)
    PS = (W + L.transpose(1, 2)) / (W + W.transpose(1, 2) + + L + L.transpose(1, 2) + 1e-8)
    PS = PS.numpy()
    PSs.append(PS)

In [ ]:
# compute per-cluster log-likelihoods
def pref_loglik(P, n_clusters, assignments):
    loglik = []
    for c in range(n_clusters):
        cluster_filt = (assignments == c)
        y_clust = y[cluster_filt].numpy()
        p_clust = P[c, item_j_indices[cluster_filt].numpy(), item_i_indices[cluster_filt].numpy()]
        loglik.append((y_clust * np.log(p_clust + 1e-8) + (1 - y_clust) * np.log(1 - p_clust + 1e-8)).sum())
    return np.array(loglik) 

ll0s = []
ll1s = []
llSs = []
for n_clusters, assignments, P0, P1, PS in zip(ALL_N_CLUSTERS, all_assignments, P0s, P1s, PSs):
    ll0 = pref_loglik(P0, n_clusters, assignments)
    ll1 = pref_loglik(P1, n_clusters, assignments)
    llS = pref_loglik(PS, n_clusters, assignments)
    ll0s.append(ll0)
    ll1s.append(ll1)
    llSs.append(llS)

ll0s[-1].mean(), ll1s[-1].mean(), llSs[-1].mean()

In [ ]:
# estimate 95% chi-squared threshold
from scipy.stats import chi2
chi2_threshold_0v1 = chi2.ppf(0.95, df=1)
chi2_threshold_0vS = chi2.ppf(0.95, df=m*(m-3)/2)
chi2_threshold_0v1, chi2_threshold_0vS

In [ ]:
# compute per-cluster likelihood ratio statistics
lr_0v1s = []
lr_0vSs = []
for ll0, ll1, lls in zip(ll0s, ll1s, llSs):
    lr_0v1 = 2 * (ll1 - ll0)
    lr_0vS = 2 * (lls - ll0)
    lr_0v1s.append(lr_0v1)
    lr_0vSs.append(lr_0vS)
(lr_0v1s[-1] > chi2_threshold_0v1).mean(), (lr_0vSs[-1] > chi2_threshold_0vS).mean()

In [ ]:
T_0vS_color = "#7EC8E3"
T_0v1_color = "#FF7F0E"

In [ ]:
means = np.array([np.mean(b) for b in lr_0vSs])
ci_low = np.array([np.percentile(b, 2.5) for b in lr_0vSs])
ci_high = np.array([np.percentile(b, 97.5) for b in lr_0vSs])
yerr = np.vstack([means - ci_low, ci_high - means])

fig, ax = plt.subplots(figsize=(8, 6))

log_x = np.log10(ALL_N_CLUSTERS)
log_width = 0.1

left  = 10 ** (log_x - log_width/2)
right = 10 ** (log_x + log_width/2)
bar_widths = right - left

ax.bar(
    ALL_N_CLUSTERS,
    means,
    yerr=yerr,
    color=T_0vS_color,
    edgecolor="none",
    capsize=4,
    width=bar_widths,
)

ax.axhline(
    y=chi2_threshold_0vS,
    color=T_0vS_color,
    linestyle="--",
    linewidth=2,
    label="Critical Value (α=0.05)",
)

ax.set_xticks(ALL_N_CLUSTERS)
ax.set_xticklabels([str(c) for c in ALL_N_CLUSTERS])

ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Likelihood Ratio Statistic")
ax.set_title("Likelihood Ratio Statistics by Number of Clusters")

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.grid(axis="y", linestyle=":", alpha=0.5)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
cluster_sizes = (assignments[:, None] == torch.arange(n_clusters)).sum(dim=0).numpy()
plt.figure(figsize=(8, 6))
plt.scatter(cluster_sizes[1:], lr_0v1s[-1][1:], alpha=0.7, label="BT vs. BT+Adv [df=1]", s=8, color=T_0v1_color)
plt.scatter(cluster_sizes[1:], lr_0vSs[-1][1:], alpha=0.7, label="BT vs. Saturated [df=m(m-3)/2]", s=8, color=T_0vS_color)

# Add horizontal line for critical value
plt.axhline(y=chi2_threshold_0v1, linestyle='--', label='Critical Value (α=0.05)', color=T_0v1_color)
plt.axhline(y=chi2_threshold_0vS, linestyle='--', label='Critical Value (α=0.05)', color=T_0vS_color)
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Cluster Size (log scale)")
plt.legend()
plt.ylabel("Likelihood Ratio Test Statistic")
plt.title("Likelihood Ratio Test Statistic vs Cluster Size")
plt.grid(True, which="both", ls="--", linewidth=0.5)

In [ ]:
def bt_fisher_information_trace(
    theta: torch.Tensor,
    item_i_indices: torch.Tensor,
    item_j_indices: torch.Tensor,
) -> torch.Tensor:
    theta = theta.to(dtype=torch.float64)
    i = item_i_indices.long()
    j = item_j_indices.long()

    delta = theta[i] - theta[j]
    p = torch.sigmoid(delta)
    w = p * (1.0 - p)
    return 2.0 * w.sum()

n_clusters = ALL_N_CLUSTERS[-1]
assignments = all_assignments[-1]
model_0 = fit_cluster_bt(
    cluster_indices=torch.tensor(assignments, dtype=torch.long),
    item_i_indices=D["item_i_indices"],
    item_j_indices=D["item_j_indices"],
    y=D["y"],
    advantage=False,
)
theta = model_0.bt_item_popularity.detach()

I = []
for c in range(n_clusters):
    cluster_filt = (assignments == c)
    item_i_clust = item_i_indices[cluster_filt]
    item_j_clust = item_j_indices[cluster_filt]
    I_c = bt_fisher_information_trace(
        theta[c],
        item_i_clust,
        item_j_clust,
    )
    I.append(I_c.item())
I = np.array(I)
I.min(), np.median(I), I.max()

In [ ]:
# Plot Fisher information vs. 0 vs S likelihood ratio statistic
lr_0vS = lr_0vSs[-1]
plt.figure(figsize=(8, 6))
plt.scatter(I, lr_0vS, color=T_0vS_color, s=3, alpha=0.7)
plt.axhline(y=chi2_threshold_0vS, linestyle='--', label='Critical Value (α=0.05)', color=T_0vS_color)
plt.xlabel("Information Trace")
plt.ylabel("Likelihood Ratio Test Statistic")
plt.yscale("log")
plt.xscale("log")
plt.title("Fisher Information vs. BT vs. Saturated Likelihood Ratio")
plt.legend()
plt.show()

In [ ]:
lr_0v1 = lr_0v1s[-1]
plt.figure(figsize=(8, 6))
plt.scatter(I, lr_0v1, color=T_0v1_color, s=3, alpha=0.7)
plt.axhline(y=chi2_threshold_0v1, linestyle='--', label='Critical Value (α=0.05)', color=T_0v1_color)
plt.xlabel("Information Trace")
plt.ylabel("Likelihood Ratio Test Statistic")
plt.yscale("log")
plt.xscale("log")
plt.title("Fisher Information vs. BT vs. BT+Adv Likelihood Ratio")
plt.legend()
plt.show()

In [ ]:
n_clusters = ALL_N_CLUSTERS[-1]
I = I
lr_0v1 = lr_0v1s[-1]
lr_0vS = lr_0vSs[-1]
filt = (I >= 1) & (lr_0v1 <= chi2_threshold_0v1) & (lr_0vS <= 1.2)
np.save(FILTER_PATH, filt)
print(f"Filtering to {filt.sum()} homogeneous clusters, from {n_clusters} total")

In [ ]:
from misc import TOPICS_PATH
import json

topics = [""] * n_clusters
with open(TOPICS_PATH, "r") as f:
    for line in f:
        entry = json.loads(line)
        cluster_idx = entry["gid"]
        topic = entry["topic"]
        topics[cluster_idx] = topic

np.random.seed(0)
homogeneous_cluster_indices = np.where(filt)[0]
heterogeneous_cluster_indices = np.where(~filt)[0]
sampled_homogeneous_cluster_indices = np.random.choice(
    homogeneous_cluster_indices,
    size=min(10, len(homogeneous_cluster_indices)),
    replace=False,
)
sampled_heterogeneous_cluster_indices = np.random.choice(
    heterogeneous_cluster_indices,
    size=min(10, len(heterogeneous_cluster_indices)),
    replace=False,
)

print(f"Sampled homogeneous cluster topics: {[topics[i] for i in sampled_homogeneous_cluster_indices]}")
print(f"Sampled heterogeneous cluster topics: {[topics[i] for i in sampled_heterogeneous_cluster_indices]}")

In [ ]:
from scipy.special import expit
from sklearn.manifold import MDS

def expected_conditional_J(theta_p, theta_q):
    """Expected conditional Jeffreys divergence between two BT models."""
    Lp = theta_p[:, None] - theta_p[None, :]
    Lq = theta_q[:, None] - theta_q[None, :]
    P = expit(Lp)
    Q = expit(Lq)
    J = (P - Q) * (Lp - Lq)
    iu = np.triu_indices_from(J, k=1)
    return np.mean(J[iu])

def pairwise_divergence_matrix(thetas):
    C = len(thetas)
    D = np.zeros((C, C))
    for i in range(C):
        for j in range(i + 1, C):
            d = expected_conditional_J(thetas[i], thetas[j])
            D[i, j] = D[j, i] = d
    return D

def embed_clusters_from_divergence(D, n_components=2, random_state=0):
    mds = MDS(n_components=n_components, dissimilarity='precomputed',
              random_state=random_state)
    return mds.fit_transform(D)   # (C, 2)

def plot_cluster_embedding(coords, topics):
    coords = np.asarray(coords)
    x, y = coords[:, 0], coords[:, 1]

    _, ax = plt.subplots(figsize=(16, 8))

    # Points with your theme color
    ax.scatter(x, y, s=60, alpha=0.9,
               edgecolors="#3A4A5A", linewidths=0.8,
               color="#7EC8E3")

    # Vertical offset so text is above points, not on top
    y_span = y.max() - y.min() if y.max() > y.min() else 1.0
    y_off = 0.015 * y_span  # 0.8% of span

    for i, txt in enumerate(topics):
        ax.text(
            x[i], y[i] + y_off,
            txt,
            fontsize=12,
            ha="center",
            va="bottom",
        )

    ax.set_title("Cluster Embedding from Jeffreys Divergence", fontsize=14, pad=10)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_frame_on(False)
    plt.tight_layout()
    plt.show()

bt_params = model_0.bt_item_popularity.detach().cpu().numpy()[homogeneous_cluster_indices]
D = pairwise_divergence_matrix(bt_params)
coords = embed_clusters_from_divergence(D)
plot_cluster_embedding(coords, [topics[c] for c in homogeneous_cluster_indices])

In [ ]:
from misc import load_embeddings

emb = load_embeddings()
centers = []
for c in homogeneous_cluster_indices:
    center = emb[assignments == c].mean(axis=0)
    centers.append(center)

x = []
y = []
for i in range(len(centers)):
    for j in range(i + 1, len(centers)):
        sim = F.cosine_similarity(
            torch.tensor(centers[i]).unsqueeze(0),
            torch.tensor(centers[j]).unsqueeze(0),
        ).item()
        x.append(D[i, j])
        y.append(sim)

In [ ]:
# Compute R squared
x_np = np.array(x)
y_np = np.array(y)
correlation_matrix = np.corrcoef(x_np, y_np)
correlation_xy = correlation_matrix[0, 1]
r_squared = correlation_xy**2
print(f"R squared between divergence and embedding similarity: {r_squared:.4f}")

In [ ]:
# Plot divergence vs. embedding similarity
plt.figure(figsize=(8, 6))
plt.scatter(x, y, color="green", s=5, alpha=0.7)

# Add R squared annotation
plt.text(0.05, 0.95, f"$R^2$ = {r_squared:.4f}", transform=plt.gca().transAxes,
         fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.5))

plt.xlabel("Expected Conditional Jeffreys Divergence")
plt.ylabel("Embedding Cosine Similarity")
plt.title("Cluster Divergence vs. Embedding Similarity")
plt.grid(True, which="both", ls="--", linewidth=0.5)
plt.show()